In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd
import dask

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update('jax_log_compiles', True)
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset, load_dataset_scheme, Dataset
from lib.ehr.interface import Patients
from lib.ehr.concepts import CPRDDemographicVectorConfig, DemographicVectorConfig


In [3]:
import logging
logging.root.level = logging.DEBUG

In [4]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")
cache_to_disk = None #'cached_inteface/m4inpatients_8000'
use_cached =  False # 'cached_inteface/m4inpatients_8000' #cache_to_disk        # False # 
cprd_interface_scheme = dict(dx='DxLTC212FlatCodes',
                             outcome='dx_cprd_ltc212',
#                              outcome='dx_cprd_ltc9809',
                            ethnicity='CPRDEthnicity5')

m3_interface_scheme = dict(dx='DxICD9',
#                            dx='DxCCS',
                           outcome='dx_flatccs_filter_v1',
#                            outcome='dx_icd9_filter_v3_groups',
                           ethnicity='MIMIC4Eth5'
#                            ethnicity='MIMIC3Eth37'
                          )

In [ ]:
if use_cached:
    cprd_patients = Patients.load(use_cached)
    splits = cprd_patients.dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')

else:
    with U.modified_environ(DATA_DIR=DATA_DIR), dask.config.set(scheduler='processes', num_workers=12):
        
        # Load dataset
        cprd_dataset = load_dataset('M4', sample=1000)
        # Use training-split for fitting the outlier_remover and the scalers.
        splits = cprd_dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')

        # Demographic vector attributes
        demographic_vector_conf = DemographicVectorConfig(age=True,
                                                          gender=True,
                                                          ethnicity=True)
        # Load interface
        cprd_patients = Patients(cprd_dataset, demographic_vector_conf,
                                **m3_interface_scheme).load_subjects(num_workers=12)

        # Cache to disk
#         cprd_patients.save(cache_to_disk, overwrite=True)

DEBUG:root:Loading dataframe files
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4-cohort/adm_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4-cohort/dx_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4-cohort/static_df.csv


In [13]:
len(cprd_patients.subjects)

999

In [14]:
from lib.ml import (ICENODE, ICENODEDimensions, OutpatientEmbeddingDimensions, 
                    Trainer, TrainerReporting, OptimizerConfig, WarmupConfig)
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC,
                      CodeGroupTopAlarmAccuracy, LossMetric, ObsCodeLevelLossMetric)

import jax.random as jrandom

In [15]:
emb_dims = OutpatientEmbeddingDimensions(dx=30, demo=5)
dims = ICENODEDimensions(mem=15, emb=emb_dims)
key = jrandom.PRNGKey(0)

m = ICENODE(dims=dims, 
              scheme=cprd_patients.dataset.scheme,
              demographic_vector_config=cprd_patients.demographic_vector_config,
              key=key)

In [16]:
splits = cprd_patients.random_splits([0.9, 0.95], 
                                    balanced='admissions')

trainer = Trainer(optimizer_config=OptimizerConfig(opt='adam', lr=1e-3),
                    reg_hyperparams=None,
                    epochs=80,
                    batch_size=128,
                    dx_loss='balanced_focal_softmax_bce')

warmup = WarmupConfig(epochs=0.1, 
                      batch_size=8,
                      opt='adam', lr=1e-3, 
                      decay_rate=0.5)

loss_metric =  LossMetric(cprd_patients, 
                          dx_loss=('softmax_bce', 'balanced_focal_softmax_bce', 
                                   'balanced_focal_bce', 'allpairs_exp_rank', 'allpairs_hard_rank', 
                                   'allpairs_sigmoid_rank'))

metrics = [CodeAUC(cprd_patients), 
           AdmissionAUC(cprd_patients), 
           CodeGroupTopAlarmAccuracy(cprd_patients, n_partitions=5, 
                                     top_k_list=[3, 5, 10, 15, 20],
                                     train_split=splits[0]), 
           loss_metric]


reporting = TrainerReporting(output_dir='dx_icenode',
                             metrics=metrics,
                             console=True,
                             parameter_snapshots=True,
                             config_json=True)

In [17]:
res = trainer(m, cprd_patients, 
              splits=splits,
              reporting=reporting,
              n_evals=100,
              warmup_config=warmup,
              continue_training=False)

INFO:root:Warming up...


Loading to device: 0subject [00:00, ?subject/s]

  0%|          | 0/1 [00:00<?, ?Epoch/s]

DEBUG:jax._src.interpreters.pxla:Compiling _shuffle for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(int32[89])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]


  0%|          | 0/47 [00:00<?, ?Batch/s]

...

DEBUG:jax._src.interpreters.pxla:Compiling integer_pow for with global shapes and types [ShapedArray(float32[])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]


Loading to device:   0%|          | 0/37 [00:00<?, ?subject/s]

Embedding:   0%|          | 0/37 [00:00<?, ?subject/s]

  0%|          | 0.00/42205.59 [00:00<?, ?longitudinal-days/s]

DEBUG:jax._src.interpreters.pxla:Compiling integer_pow for with global shapes and types [ShapedArray(float32[])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]


In [7]:
import copy
class A:
    def __init__(self, x, y):
        self.x = x
        self.y = y

a = A(2, 4)
b = copy.copy(a)
b.x = 6
b.y = 0
a.x

2

In [11]:
a.__class__.__name__

'A'